In [1]:
import os
import pandas as pd
import numpy as np
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/mu529/Bus-Capstone')
import ttools #homemade module
import gtfs #homemade module
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')
%matplotlib inline

In [2]:
# get all the schedule data. (subset can be created later)
trips = gtfs.load_trips('gtfs/')
stops = gtfs.load_stops('gtfs/')
stop_times, tz_sched = gtfs.load_stop_times('gtfs/')
tcal = gtfs.TransitCalendar('gtfs/')
print 'Finished loading GTFS data.'

Finished loading GTFS data.


In [3]:
# get the sample of parsed AVL data.  Beware, large files take more time.
bustime = pd.read_csv('newdata_parsed.csv')#,parse_dates=dt_columns)
bustime.drop_duplicates(['vehicleID','RecordedAtTime'],inplace=True)
bustime['Trip'] = bustime['Trip'].str.replace('MTA NYCT_','')
bustime.set_index(['Line','Trip','TripDate','vehicleID','RecordedAtTime'],
                  inplace=True,drop=True,verify_integrity=True)

In [4]:
# for demonstration, use a subset. Just get data for one line (M5) on one day.
tripDateLookup = "2016-06-13"
bustime = bustime.xs((tripDateLookup),level=(2),drop_level=False)
# note that the AVL dataframe must be sorted by timestammp, since iloc[]
# selection is used later in this script to find the earliest time
bustime.sort_index(inplace=True)
print 'Finished loading BusTime data and and slicing one day.'

Finished loading BusTime data and and slicing one day.


In [11]:
active_services = tcal.get_service_ids('2016-06-13')
trips_ = gtfs.load_trips('gtfs/').reset_index().set_index(['service_id'])
gtfs_trip_ids = trips_.loc[active_services]['trip_id'].unique()

In [13]:
bustime_trip_ids = bustime.index.get_level_values(1).unique()
bustime_trip_ids[:5]

array(['FB_B6-Weekday-SDon-044500_B49_21',
       'FB_B6-Weekday-SDon-047300_B49_15',
       'FB_B6-Weekday-SDon-047400_B49_21',
       'FB_B6-Weekday-SDon-051000_B49_15',
       'FB_B6-Weekday-SDon-051200_B49_21'], dtype=object)

In [12]:
gtfs_trip_ids[:5]

array(['GH_B6-Weekday-SDon-004000_BX12_1',
       'GH_B6-Weekday-SDon-009000_BX12_1',
       'GH_B6-Weekday-SDon-013000_BX12_1',
       'GH_B6-Weekday-SDon-017000_BX12_1',
       'GH_B6-Weekday-SDon-021000_BX12_1'], dtype=object)

In [15]:
len(gtfs_trip_ids)

67414

In [17]:
len(bustime_trip_ids)

52346

## Hmmm there appears to be a big difference in the trip_id lists.
Return the non-matching trip_ids and let's examine the data from the GTFS import.

THEY ALL APPEAR TO BE FROM MTABC DATA.  (Perhaps the BusTime sample didn't include those buses?)

In [19]:
set1 = set(gtfs_trip_ids)
set2 = set(bustime_trip_ids)

unmatched = set1.symmetric_difference(set2) # set(['pig'])

In [21]:
trips.loc[unmatched]

,block_id,direction_id,route_id,service_id,shape_id,trip_headsign
trip_id,,,,,,
MTABC_12073125-YOPB6-YO_B6-Weekday-10,NaN,NaN,NaN,NaN,NaN,NaN
MTABC_12060838-LGPB6-LG_B6-Weekday-10-SDon,NaN,NaN,NaN,NaN,NaN,NaN
12034804-JKPB6-JK_B6-Weekday-10-SDon,2354785,1,Q10,JKPB6-JK_B6-Weekday-10-SDon,Q100465,LIMITED JFK AIRPORT CENTRAL TERMINAL via LEFFE...
GA_B6-Weekday-107400_B47_430,NaN,0,B47,GA_B6-Weekday,B470075,BED-STUY WOODHULL HOSP via RALPH AV
FB_B6-Weekday-054300_B41_1,NaN,0,B41,FB_B6-Weekday,B410310,LTD DNTWN BKLYN CADMAN PLZ via FLATBUSH
FP_B6-Weekday-053800_Q54_107,NaN,0,Q54,FP_B6-Weekday,Q54C0085,JAMAICA 170 ST
FB_B6-Weekday-046800_SB44_507,NaN,0,B44+,FB_B6-Weekday,SB44C0035,SBS FLUSHING AV via NOSTRAND via ROGERS
MTABC_11761792-BPPB6-BP_B6-Weekday-10-SDon,NaN,NaN,NaN,NaN,NaN,NaN
MTABC_12034707-JKPB6-JK_B6-Weekday-10-SDon,NaN,NaN,NaN,NaN,NaN,NaN
